In [1]:
import numpy as np
import pandas as pd
from math import log

In [13]:
def create_data():
    datasets = [['young', 'no', 'no', 'fair', 'no'],
               ['young', 'no', 'no', 'good', 'no'],
               ['young', 'yes', 'no', 'good', 'yes'],
               ['young', 'yes', 'yes', 'fair', 'yes'],
               ['young', 'no', 'no', 'fair', 'no'],
               ['middle-aged', 'no', 'no', 'fair', 'no'],
               ['middle-aged', 'no', 'no', 'good', 'no'],
               ['middle-aged', 'yes', 'yes', 'good', 'yes'],
               ['middle-aged', 'no', 'yes', 'excellent', 'yes'],
               ['middle-aged', 'no', 'yes', 'excellent', 'yes'],
               ['senior', 'no', 'yes', 'excellent', 'yes'],
               ['senior', 'no', 'yes', 'good', 'yes'],
               ['senior', 'yes', 'no', 'good', 'yes'],
               ['senior', 'yes', 'no', 'excellent', 'yes'],
               ['senior', 'no', 'no', 'fair', 'no'],
               ]
    labels = ['age', 'employed', 'homeowner', 'credit rating', 'class']
    return datasets, labels

In [14]:
datasets, labels = create_data()

In [15]:
train_data = pd.DataFrame(datasets, columns=labels)

In [16]:
train_data

,age,employed,homeowner,credit rating,class
0,young,no,no,fair,no
1,young,no,no,good,no
2,young,yes,no,good,yes
3,young,yes,yes,fair,yes
4,young,no,no,fair,no
5,middle-aged,no,no,fair,no
6,middle-aged,no,no,good,no
7,middle-aged,yes,yes,good,yes
8,middle-aged,no,yes,excellent,yes
9,middle-aged,no,yes,excellent,yes


In [17]:
def calc_ent(datasets):
    data_length = len(datasets)
    label_count = {}
    for i in range(data_length):
        label = datasets[i][-1]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
    ent = -sum([(p/data_length)*log(p/data_length, 2) for p in label_count.values()])
    return ent

def cond_ent(datasets, axis=0):
    data_length = len(datasets)
    feature_sets = {}
    for i in range(data_length):
        feature = datasets[i][axis]
        if feature not in feature_sets:
            feature_sets[feature] = []
        feature_sets[feature].append(datasets[i])
    cond_ent = sum([(len(p)/data_length)*calc_ent(p) for p in feature_sets.values()])
    return cond_ent

def info_gain(ent, cond_ent):
    return ent - cond_ent

def info_gain_train(datasets):
    count = len(datasets[0]) - 1
    ent = calc_ent(datasets)
    best_feature = []
    for c in range(count):
        c_info_gain = info_gain(ent, cond_ent(datasets, axis=c))
        best_feature.append((c, c_info_gain))
        print('Feature ({}) - info_gain - {:.3f}'.format(labels[c], c_info_gain))
    best_ = max(best_feature, key=lambda x: x[-1])
    return 'Feature ({}) has the highest information gain, selected as the root node feature'.format(labels[best_[0]])

In [18]:
info_gain_train(np.array(datasets))

Feature (age) - info_gain - 0.083
Feature (employed) - info_gain - 0.324
Feature (homeowner) - info_gain - 0.420
Feature (credit rating) - info_gain - 0.363


'Feature (homeowner) has the highest information gain, selected as the root node feature'

In [19]:
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {'label:': self.label, 'feature': self.feature, 'tree': self.tree}

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)

class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}

    @staticmethod
    def calc_ent(datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p/data_length)*log(p/data_length, 2) for p in label_count.values()])
        return ent

    def cond_ent(self, datasets, axis=0):
        data_length = len(datasets)
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = []
            feature_sets[feature].append(datasets[i])
        cond_ent = sum([(len(p)/data_length)*self.calc_ent(p) for p in feature_sets.values()])
        return cond_ent

    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent

    def info_gain_train(self, datasets):
        count = len(datasets[0]) - 1
        ent = self.calc_ent(datasets)
        best_feature = []
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(datasets, axis=c))
            best_feature.append((c, c_info_gain))
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_

    def train(self, train_data):
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[:-1]
        if len(y_train.value_counts()) == 1:
            return Node(root=True,
                        label=y_train.iloc[0])

        if len(features) == 0:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]

        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        node_tree = Node(root=False, feature_name=max_feature_name, feature=max_feature)

        feature_list = train_data[max_feature_name].value_counts().index
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)

            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)

In [20]:
datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)
dt = DTree()
tree = dt.fit(data_df)

In [21]:
tree

{'label:': None, 'feature': 2, 'tree': {'no': {'label:': None, 'feature': 1, 'tree': {'no': {'label:': 'no', 'feature': None, 'tree': {}}, 'yes': {'label:': 'yes', 'feature': None, 'tree': {}}}}, 'yes': {'label:': 'yes', 'feature': None, 'tree': {}}}}

In [24]:
dt.predict(['young', 'yes', 'no', 'fair'])

'yes'